## Scraping Detik

In [53]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [71]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [72]:
jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'ganjar'

In [73]:
def scrape_links(page_number,keywords):
    url = f"https://www.detik.com/search/searchall?query={keywords}&sortby=time&page={page_number}"
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [74]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


https://www.detik.com/search/searchall?query=ganjar&sortby=time&page=1
Scraped 9 links from page 1
Total Links: 9


In [75]:
def scrape_url(url,keywords):
    response = requests.get(url)
    if response.status_code == 200:
        # title =[]
        # author = []
        # date = []
        # content = []
        # category =[]
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1', {"class": "detail__title"})
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "detail__author"})
        if author_elem:
            author_text = author_elem.get_text()
            author_text = author_text.split('-')[0].strip()
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "detail__date"})
        if date_elem:
            date_text = date_elem.text.strip()
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('div',{"class": "page__breadcrumb"})
        if category_elements:
            category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
            category_text= category_text.text.strip()
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('div', {"class": "detail__body"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'keywords':keywords,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [76]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [77]:
df = pd.DataFrame(results)

df.head(10)

,title,keywords,author,category,date,content,link
0,"Jadi Ketua TPN Ganjar, Arsjad Rasjid Cuti dari...",ganjar,Marlinda Oktavia Erwanti,Pemilu,"Selasa, 26 Sep 2023 09:25 WIB",Ketua Tim Pemenangan Nasional (TPN) bakal calo...,https://news.detik.com/pemilu/d-6950733/jadi-k...
1,"Jokowi: Beda Pilihan itu Wajar, Mau Pilih Prab...",ganjar,Indra Komara,Pemilu,"Selasa, 26 Sep 2023 17:24 WIB",Presiden Joko Widodo (Jokowi) mengingatkan kad...,https://news.detik.com/pemilu/d-6951804/jokowi...
2,Projo Bakal Dukung Capres yang Depannya P,ganjar,Tara Wahyu NV,Pemilu,"Selasa, 26 Sep 2023 15:29 WIB",Relawan pendukung Presiden Joko Widodo (Jokowi...,https://news.detik.com/pemilu/d-6951509/projo-...
3,Menerka Sosok Cawapres Ganjar Jika Megawati Ta...,ganjar,Kadek Melda Luxiana,Pemilu,"Selasa, 26 Sep 2023 08:45 WIB","PPP meminta Ketum PDIP, Megawati Soekaroputri ...",https://news.detik.com/pemilu/d-6950710/menerk...
4,Soal Hubungan Megawati dan Jokowi Usai Kaesang...,ganjar,Tim detikcom,Pemilu,"Selasa, 26 Sep 2023 09:03 WIB","Putra Presiden Joko Widodo (Jokowi), Kaesang P...",https://news.detik.com/pemilu/d-6950719/soal-h...
5,BEM FISIP Se-Surabaya Tantang 3 Capres Adu Gag...,ganjar,Esti Widiyana,Berita,"Selasa, 26 Sep 2023 15:46 WIB",Badan Eksekutif Mahasiswa Fakultas Ilmu Sosial...,https://www.detik.com/jatim/berita/d-6951552/b...
6,Ketum Projo Bocorkan Nama Capres yang Akan Did...,ganjar,Tara Wahyu NV,Berita,"Selasa, 26 Sep 2023 14:51 WIB","Projo, organisasi pendukung Presiden Joko Wido...",https://www.detik.com/jateng/berita/d-6951412/...
7,Harapan Besar Pecinta Budaya Populer Indonesia...,ganjar,Atmi Ahsani Yusron,Berita Kpop,"Selasa, 26 Sep 2023 14:24 WIB",Muda-mudi Indonesia sedang bergelora dua tahun...,https://hot.detik.com/berita-kpop/d-6951328/ha...
8,"Jadi Ketua TPN Ganjar Pranowo, Arsjad Rasjid C...",ganjar,Anisa Indraini,Berita Ekonomi Bisnis,"Selasa, 26 Sep 2023 09:10 WIB",Ketua Umum Kamar Dagang dan Industri (KADIN) I...,https://finance.detik.com/berita-ekonomi-bisni...


Save Data

In [78]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/detik_{keywords}_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

In [79]:
# links=[]
# # jumlah Page yang diambil 
# jumlah_index = 20

# for index in range(jumlah_index):
#     url = f"https://news.detik.com/indeks/{index+1}"  
#     # print(url)
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     articles = soup.find_all('article', {"class": "list-content__item"})
    
#     for article in articles:
#         link = article.find('a')['href']
#         links.append(link)
    
# print("Total Links",len(links))

In [80]:
# title = []
# author = []
# date = []
# content = []
# category =[]

# for link in links:
#     url = link
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Judul Berita
#     title_elem = soup.find('h1', {"class": "detail__title"})
#     if title_elem:
#         title_text = title_elem.text.strip()
#     else:
#         title_text = "Title not found"
#     title.append(title_text)
    
#     # Author berita
#     author_elem = soup.find('div', {"class": "detail__author"})
#     if author_elem:
#         author_text = author_elem.get_text()
#         author_text = author_text.split('-')[0].strip()
#     else:
#         author_text = "Author not found"
#     author.append(author_text)
    
#     # tanggal berita
#     date_elem = soup.find('div', {"class": "detail__date"})
#     if date_elem:
#         date_text = date_elem.text.strip()
#     else:
#         date_text = "Date not found"
#     date.append(date_text)
    
#     # Category berita
#     category_elements = soup.find('div',{"class": "page__breadcrumb"})
#     if category_elements:
#         category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
#         category_text= category_text.text.strip()
#     else:
#         category_text = "Category not found"
#     category.append(category_text)
   
    
#     # Content Berita
#     body_elem = soup.find('div', {"class": "detail__body"})
    
#     if body_elem:
#         content_elem = body_elem.find_all('p')
#         content_text = ""
#         for p in content_elem:
#             content_text += p.text.strip() + "\n"
        
#         if content_text.strip():
#             content.append(content_text)
#         else:
#             content.append("Content not found")
#     else:
#         content.append("Content not found")
    
    

In [81]:
# data = {'title': title,
#         'author' : author,
#         'category':category,
#         'date': date,
#         'content' : content,
#         'link' : links
#         }

# df = pd.DataFrame(data)

# df.head(10)

In [82]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/detik_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/detik_2023-09-26_17-27-53.xlsx


In [83]:
# # Latihan Pengecek dan find element
# url = f"https://news.detik.com/detiknews/d-6948823/siskaeee-penuhi-panggilan-polisi-soal-film-porno-ini-penampilannya"  
# # print(url)
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")


# Category
# category_elements = soup.find('div',{"class": "page__breadcrumb"})
# print(category_elements)
# category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
# print(category_text.text.strip())

# ambil author detik
# title_elem = soup.find('div', {"class": "detail__author"})
# author_text = title_elem.get_text()
# author_name = author_text.split('-')[0].strip()
# print(author_name)

In [84]:
# def scrape_url(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#     else:
#         print(f"Failed to retrieve data from {url}")

In [85]:
# urls = ['https://kompas.com', 'https://kompas.com', 'https://kompas.com']
# threads = []

In [86]:
# for url in urls:
#     thread = threading.Thread(target=scrape_url, args=(url,))
#     thread.start()
#     threads.append(thread)

# print(threads)

In [87]:
# for thread in threads:
#     thread.join()
    
# print("All threads finished")